# Optimización de Memoria del DataFrame

Este notebook se centra en la optimización del uso de memoria del DataFrame meteorológico, manteniendo la integridad de los datos mientras se reduce significativamente su uso de memoria.

## 1. Carga y Análisis Inicial del DataFrame

Primero, cargaremos el DataFrame y analizaremos su uso actual de memoria, tipos de datos y valores únicos en cada columna.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import sys

# Función para mostrar el uso de memoria
def memoria_df(df):
    memoria = df.memory_usage(deep=True).sum()
    print(f'Uso de memoria: {memoria / 1024**2:.2f} MB')
    return memoria

# Cargar el DataFrame
df = pd.read_parquet("../data_parquet/meteo")

# Mostrar información inicial
print("Información inicial del DataFrame:")
df.info()
print("\nUso de memoria inicial:")
memoria_inicial = memoria_df(df)

# Analizar valores únicos por columna
print("\nNúmero de valores únicos por columna:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()}")

Información inicial del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339376 entries, 0 to 2339375
Data columns (total 24 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   time                   datetime64[ns]
 1   temperature_2m         float64       
 2   precipitation          float64       
 3   rain                   float64       
 4   cloud_cover            int64         
 5   cloud_cover_low        int64         
 6   cloud_cover_mid        int64         
 7   cloud_cover_high       int64         
 8   wind_speed_10m         float64       
 9   wind_speed_100m        float64       
 10  wind_direction_10m     int64         
 11  wind_direction_100m    int64         
 12  wind_gusts_10m         float64       
 13  date                   object        
 14  hour                   object        
 15  latitude               float64       
 16  longitude              float64       
 17  generationtime_ms      float64

## 2. Optimización de Columnas Numéricas

Vamos a optimizar las columnas numéricas reduciendo su precisión cuando sea posible y utilizando tipos de datos más eficientes.

In [2]:
# Función para reducir la memoria de columnas numéricas
def optimizar_tipos_numericos(df):
    df_optimizado = df.copy()
    
    for columna in df.select_dtypes(include=['float64']).columns:
        # Convertir a float32 si no hay pérdida significativa de precisión
        valores_min = df[columna].min()
        valores_max = df[columna].max()
        if valores_min >= np.finfo('float32').min and valores_max <= np.finfo('float32').max:
            df_optimizado[columna] = df[columna].astype(np.float32)
    
    for columna in df.select_dtypes(include=['int64']).columns:
        # Determinar el tipo de entero más eficiente
        col_min = df[columna].min()
        col_max = df[columna].max()
        
        if col_min >= np.iinfo(np.int8).min and col_max <= np.iinfo(np.int8).max:
            df_optimizado[columna] = df[columna].astype(np.int8)
        elif col_min >= np.iinfo(np.int16).min and col_max <= np.iinfo(np.int16).max:
            df_optimizado[columna] = df[columna].astype(np.int16)
        elif col_min >= np.iinfo(np.int32).min and col_max <= np.iinfo(np.int32).max:
            df_optimizado[columna] = df[columna].astype(np.int32)
    
    return df_optimizado

# Optimizar tipos numéricos
df_optimizado = optimizar_tipos_numericos(df)

print("Uso de memoria después de optimizar tipos numéricos:")
memoria_numericos = memoria_df(df_optimizado)
print(f"\nReducción de memoria: {(memoria_inicial - memoria_numericos) / 1024**2:.2f} MB")
print("\nNuevos tipos de datos:")
print(df_optimizado.dtypes)

Uso de memoria después de optimizar tipos numéricos:
Uso de memoria: 724.01 MB

Reducción de memoria: 187.65 MB

Nuevos tipos de datos:
time                     datetime64[ns]
temperature_2m                  float32
precipitation                   float32
rain                            float32
cloud_cover                        int8
cloud_cover_low                    int8
cloud_cover_mid                    int8
cloud_cover_high                   int8
wind_speed_10m                  float32
wind_speed_100m                 float32
wind_direction_10m                int16
wind_direction_100m               int16
wind_gusts_10m                  float32
date                             object
hour                             object
latitude                        float32
longitude                       float32
generationtime_ms               float32
utc_offset_seconds                int16
timezone                         object
timezone_abbreviation            object
elevation               

## 3. Conversión de Columnas Objeto a Categorías

Vamos a convertir las columnas de tipo objeto que tienen un número limitado de valores únicos a tipo categórico.

In [3]:
# Convertir columnas objeto a categorías cuando sea apropiado
columnas_categoria = ['timezone', 'timezone_abbreviation', 'city']
for col in columnas_categoria:
    df_optimizado[col] = df_optimizado[col].astype('category')

print("Uso de memoria después de convertir columnas a categorías:")
memoria_categorias = memoria_df(df_optimizado)
print(f"\nReducción adicional de memoria: {(memoria_numericos - memoria_categorias) / 1024**2:.2f} MB")

Uso de memoria después de convertir columnas a categorías:
Uso de memoria: 334.65 MB

Reducción adicional de memoria: 389.35 MB


## 4. Optimización de Columnas de Fecha y Hora

Optimizaremos las columnas relacionadas con fechas y horas, eliminando redundancias.

In [4]:
# Eliminar columnas redundantes de fecha y hora ya que tenemos 'time'
df_optimizado = df_optimizado.drop(['date', 'hour'], axis=1)

print("Uso de memoria después de optimizar fechas:")
memoria_fechas = memoria_df(df_optimizado)
print(f"\nReducción adicional de memoria: {(memoria_categorias - memoria_fechas) / 1024**2:.2f} MB")

Uso de memoria después de optimizar fechas:
Uso de memoria: 138.33 MB

Reducción adicional de memoria: 196.33 MB


## 5. Validación de la Reducción de Memoria

Comparemos el uso de memoria antes y después de la optimización y verifiquemos la integridad de los datos.

In [5]:
# Mostrar resumen de la reducción de memoria
print("Resumen de la optimización de memoria:")
print(f"Memoria inicial: {memoria_inicial / 1024**2:.2f} MB")
print(f"Memoria final: {memoria_fechas / 1024**2:.2f} MB")
print(f"Reducción total: {(memoria_inicial - memoria_fechas) / 1024**2:.2f} MB")
print(f"Porcentaje de reducción: {((memoria_inicial - memoria_fechas) / memoria_inicial * 100):.2f}%")

# Verificar la integridad de los datos
print("\nVerificación de la integridad de los datos:")
print("\nColumnas en el DataFrame optimizado:")
print(df_optimizado.dtypes)
print("\nDimensiones del DataFrame:")
print(f"Original: {df.shape}")
print(f"Optimizado: {df_optimizado.shape}")

# Verificar que los valores numéricos sean consistentes
columnas_float = df_optimizado.select_dtypes(include=['float32', 'float64']).columns
for col in columnas_float:
    diff = np.abs(df[col] - df_optimizado[col]).max()
    print(f"\nMáxima diferencia en {col}: {diff}")

Resumen de la optimización de memoria:
Memoria inicial: 911.66 MB
Memoria final: 138.33 MB
Reducción total: 773.33 MB
Porcentaje de reducción: 84.83%

Verificación de la integridad de los datos:

Columnas en el DataFrame optimizado:
time                     datetime64[ns]
temperature_2m                  float32
precipitation                   float32
rain                            float32
cloud_cover                        int8
cloud_cover_low                    int8
cloud_cover_mid                    int8
cloud_cover_high                   int8
wind_speed_10m                  float32
wind_speed_100m                 float32
wind_direction_10m                int16
wind_direction_100m               int16
wind_gusts_10m                  float32
latitude                        float32
longitude                       float32
generationtime_ms               float32
utc_offset_seconds                int16
timezone                       category
timezone_abbreviation          category
elevati

## 6. Guardar DataFrame Optimizado

Guardaremos el DataFrame optimizado en formato parquet con la configuración óptima de compresión.

In [6]:
from pathlib import Path
import shutil
import os

# Crear directorio para los datos optimizados
output_dir = Path("../data_parquet_optimized/meteo")
if output_dir.exists():
    shutil.rmtree(output_dir)

# Guardar DataFrame optimizado
df_optimizado.to_parquet(
    "../data_parquet_optimized/meteo",
    partition_cols=['year'],
    index=False,
    compression='snappy'  # 'snappy' ofrece buen balance entre compresión y velocidad
)

print("DataFrame optimizado guardado exitosamente.")

# Verificar el tamaño del archivo en disco
def get_dir_size(path):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total

original_size = get_dir_size("../data_parquet/meteo")
optimized_size = get_dir_size("../data_parquet_optimized/meteo")

print(f"\nTamaño en disco:")
print(f"Original: {original_size / 1024**2:.2f} MB")
print(f"Optimizado: {optimized_size / 1024**2:.2f} MB")
print(f"Reducción en disco: {((original_size - optimized_size) / original_size * 100):.2f}%")

DataFrame optimizado guardado exitosamente.

Tamaño en disco:
Original: 46.68 MB
Optimizado: 29.90 MB
Reducción en disco: 35.96%
